In [25]:
import glob

In [26]:
out = "/home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/mutated_files"

### Align using protein Kraken2 (translated search):

Create ebov amino acid ref with NCBI taxID and align with Kraken translated search:

In [3]:
!cd /home/laura/projects/kraken2-2.1.2 \
    && kraken2-build --db kraken_ebov_protein --protein --download-taxonomy

Downloaded accession to taxon map(s)
Uncompressing taxonomy data... done.
Untarring taxonomy tree data... done.


In [4]:
# ebola_rdrp_aa.fasta contains the Ebola RdRP amino acid sequence from PalmDB (u55137) labeled with the NCBI Ebola taxonomy ID (186538)
ebov_aa_fasta = "/home/laura/projects/kraken2-2.1.2/ebola_rdrp_aa.fasta"

In [5]:
# Add amino acid ebola sequence
!cd /home/laura/projects/kraken2-2.1.2 \
    && /usr/bin/time -v \
        kraken2-build \
            --db kraken_ebov_protein \
            --protein \
            --no-masking \
            --add-to-library $ebov_aa_fasta

Added "/home/laura/projects/kraken2-2.1.2/ebola_rdrp_aa.fasta" to library (kraken_ebov_protein)
	Command being timed: "kraken2-build --db kraken_ebov_protein --protein --no-masking --add-to-library /home/laura/projects/kraken2-2.1.2/ebola_rdrp_aa.fasta"
	User time (seconds): 0.14
	System time (seconds): 0.04
	Percent of CPU this job got: 45%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.41
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 5480
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 2
	Minor (reclaiming a frame) page faults: 7120
	Voluntary context switches: 19
	Involuntary context switches: 13
	Swaps: 0
	File system inputs: 360
	File system outputs: 16
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


Note: Also tried 31mers (instead of default 15mers) which lead to no alignments.

In [6]:
# Build index
!cd /home/laura/projects/kraken2-2.1.2 \
    && /usr/bin/time -v \
        kraken2-build \
            --protein \
            --db kraken_ebov_protein \
            --build --threads 8

Creating sequence ID to taxonomy ID map (step 1)...
Sequence ID to taxonomy ID map complete. [0.023s]
Estimating required capacity (step 2)...
Estimated hash table requirement: 48272 bytes
Capacity estimation complete. [0.045s]
Building database files (step 3)...
Taxonomy parsed and converted.
CHT created with 4 bits reserved for taxid.
Completed processing of 1 sequences, 109 aa
Writing data to disk...  complete.
Database files completed. [6.964s]
Database construction complete. [Total: 7.080s]
	Command being timed: "kraken2-build --protein --db kraken_ebov_protein --build --threads 8"
	User time (seconds): 6.43
	System time (seconds): 0.98
	Percent of CPU this job got: 103%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:07.15
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 892948
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 

Align:

In [27]:
protein_krakendb = "/home/laura/projects/kraken2-2.1.2/kraken_ebov_protein"
protein_kraken_out = f"{out}/../kraken_mutations/mutations_protein"

In [28]:
# Align unmutated sequences
orig_file = "/home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698539_2_extracted_u10.fa"
out_file1 = "0_ebov_snp_0_ms.k2report"
out_file2 = "0_ebov_snp_0_ms.kraken2"

!~/bin/kraken2 \
        --db $protein_krakendb \
        --threads 32 \
        --report-minimizer-data \
        --report $protein_kraken_out/$out_file1 \
        $orig_file > $protein_kraken_out/$out_file2

Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.020s (2062.0 Kseq/m, 181.46 Mbp/m).
  636 sequences classified (94.08%)
  40 sequences unclassified (5.92%)


In [29]:
for mut_file in glob.glob(f"{out}/*_ms.fa"):
    out_file1 = mut_file.split("/")[-1].split(".fa")[0] + ".k2report"
    out_file2 = mut_file.split("/")[-1].split(".fa")[0] + ".kraken2"
    
    !~/bin/kraken2 \
            --db $protein_krakendb \
            --threads 32 \
            --report-minimizer-data \
            --report $protein_kraken_out/$out_file1 \
            $mut_file > $protein_kraken_out/$out_file2

Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.015s (2623.0 Kseq/m, 230.83 Mbp/m).
  4 sequences classified (0.59%)
  672 sequences unclassified (99.41%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.017s (2318.4 Kseq/m, 204.02 Mbp/m).
  79 sequences classified (11.69%)
  597 sequences unclassified (88.31%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.017s (2435.6 Kseq/m, 214.33 Mbp/m).
  10 sequences classified (1.48%)
  666 sequences unclassified (98.52%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.015s (2667.5 Kseq/m, 234.74 Mbp/m).
  4 sequences classified (0.59%)
  672 sequences unclassified (99.41%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.015s (2637.4 Kseq/m, 232.09 Mbp/m).
  0 sequences classified (0.00%)
  676 sequences unclassified (100.00%)
Loading database information... done.
676 sequences (0.06 Mbp) processe

### Align using DNA Kraken2:

In [11]:
# Build viral Kraken index with added EBOV genome
!cd /home/laura/projects/kraken2-2.1.2 && kraken2-build --db krakendb --download-taxonomy
!cd /home/laura/projects/kraken2-2.1.2 && kraken2-build --db krakendb --download-library viral
!cd /home/laura/projects/kraken2-2.1.2 && kraken2-build --db krakendb --add-to-library ebola_genome.fa
!cd /home/laura/projects/kraken2-2.1.2 && kraken2-build --db krakendb --build --threads 8

Downloaded accession to taxon map(s)
Uncompressing taxonomy data... done.
Untarring taxonomy tree data... done.
Step 1/2: Performing rsync file transfer of requested files
Rsync file transfer complete.
Step 2/2: Assigning taxonomic IDs to sequences
Processed 12345 projects (15404 sequences, 505.71 Mbp)... done.
All files processed, cleaning up extra sequence files... done, library complete.
Masking low-complexity regions of downloaded library...which: no dustmasker in (/home/laura/projects/kraken2-2.1.2:/home/laura/bin:/home/laura/bin:/home/laura/anaconda3/bin:/home/laura/anaconda3/condabin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/laura/.local/bin:/home/laura/bin:/home/laura/.local/bin:/home/laura/bin)
Unable to find dustmasker in path, can't mask low-complexity sequences
Masking low-complexity regions of new file...which: no dustmasker in (/home/laura/projects/kraken2-2.1.2:/home/laura/bin:/home/laura/bin:/home/laura/anaconda3/bin:/home/laura/anaconda3/condabin:/usr/loc

In [11]:
krakendb = "/home/laura/projects/kraken2-2.1.2/krakendb"
kraken_out = f"{out}/../kraken_mutations/mutations_nn"

In [30]:
# Align unmutated sequences
orig_file = "/home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698539_2_extracted_u10.fa"
out_file1 = "0_ebov_snp_0_ms.k2report"
out_file2 = "0_ebov_snp_0_ms.kraken2"

!~/bin/kraken2 \
        --db $krakendb \
        --threads 32 \
        --report-minimizer-data \
        --report $kraken_out/$out_file1 \
        $orig_file > $kraken_out/$out_file2

Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.009s (4425.0 Kseq/m, 389.40 Mbp/m).
  666 sequences classified (98.52%)
  10 sequences unclassified (1.48%)


In [31]:
for mut_file in glob.glob(f"{out}/*_ms.fa"):
    out_file1 = mut_file.split("/")[-1].split(".fa")[0] + ".k2report"
    out_file2 = mut_file.split("/")[-1].split(".fa")[0] + ".kraken2"
    
    !~/bin/kraken2 \
            --db $krakendb \
            --threads 32 \
            --report-minimizer-data \
            --report $kraken_out/$out_file1 \
            $mut_file > $kraken_out/$out_file2

Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.018s (2228.3 Kseq/m, 196.09 Mbp/m).
  6 sequences classified (0.89%)
  670 sequences unclassified (99.11%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.019s (2139.7 Kseq/m, 188.29 Mbp/m).
  210 sequences classified (31.07%)
  466 sequences unclassified (68.93%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.009s (4577.4 Kseq/m, 402.81 Mbp/m).
  33 sequences classified (4.88%)
  643 sequences unclassified (95.12%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.008s (4907.4 Kseq/m, 431.85 Mbp/m).
  16 sequences classified (2.37%)
  660 sequences unclassified (97.63%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.016s (2500.8 Kseq/m, 220.07 Mbp/m).
  0 sequences classified (0.00%)
  676 sequences unclassified (100.00%)
Loading database information... done.
676 sequences (0.06 Mbp) proces

### Align using kallisto bustools (translated search):

In [32]:
kallisto_out = f"{out}/../kallisto_mutations"
virus_t2g = "/home/laura/projects/virus-watch-data/virus_ref/nodup_clu_t2g.txt"
virus_index = "/home/laura/projects/virus-watch-data/virus_ref/kallisto_aa/noduplicates_rhesus.idx"

kallisto = "/home/laura/projects/kallisto/build/src/kallisto"
bustools = "/home/laura/projects/bustools/build/src/bustools"

In [33]:
mut_file = "/home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698539_2_extracted_u10.fa"
mut_name = "0_ebov_snp_0_ms"
    
!mkdir -p $kallisto_out/$mut_name

!$kallisto bus \
    -i $virus_index \
    -o $kallisto_out/$mut_name \
    --aa \
    -x bulk \
    -t 32 \
    $mut_file

!$bustools sort -o $kallisto_out/$mut_name/output_sorted.bus $kallisto_out/$mut_name/output.bus

!$bustools count \
    --genecounts \
    --cm \
    -o $kallisto_out/$mut_name/bustools_count/ \
    -g $virus_t2g \
    -e $kallisto_out/$mut_name/matrix.ec \
    -t $kallisto_out/$mut_name/transcripts.txt \
    $kallisto_out/$mut_name/output_sorted.bus


[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,541,842
[index] number of distinguishing flanking k-mers: 138
[quant] running in single-end mode
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698539_2_extracted_u10.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 676 reads pseudoaligned

 all fits in buffer
Read in 676 BUS records
reading time 0s
sorting time 0s
writing time 0s


In [34]:
for mut_file in glob.glob(f"{out}/*_ms.fa"):
    mut_name = mut_file.split("/")[-1].split(".fa")[0]
    
    !mkdir -p $kallisto_out/$mut_name
    
    !$kallisto bus \
        -i $virus_index \
        -o $kallisto_out/$mut_name \
        --aa \
        -x bulk \
        -t 32 \
        $mut_file

    !$bustools sort -o $kallisto_out/$mut_name/output_sorted.bus $kallisto_out/$mut_name/output.bus

    !$bustools count \
        --genecounts \
        --cm \
        -o $kallisto_out/$mut_name/bustools_count/ \
        -g $virus_t2g \
        -e $kallisto_out/$mut_name/matrix.ec \
        -t $kallisto_out/$mut_name/transcripts.txt \
        $kallisto_out/$mut_name/output_sorted.bus


[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,541,842
[index] number of distinguishing flanking k-mers: 138
[quant] running in single-end mode
[quant] will process file 1: /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/mutated_files/3_ebov_snp_18_ms.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 53 reads pseudoaligned

 all fits in buffer
Read in 53 BUS records
reading time 0s
sorting time 0s
writing time 0s

[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,541,842
[index] number of distinguishing flanking k-mers: 138
[quant] running in single-end mode
[quant] will process file 1: /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/mutated_files/2_ebov_snp_08_ms.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 335 reads pseudoaligned

 all fits in buffer
Read in 335 BUS records
reading time

### Align using kallisto bustools (standard workflow):

In [35]:
kallisto_out = f"{out}/../kallisto_mutations/nn"
ebola_t2g = "/home/laura/projects/virus-watch-data/ebola_ref/ebov_GCA_000848505_t2g.txt"
ebola_index = "/home/laura/projects/virus-watch-data/ebola_ref/ebov_GCA_000848505.idx"

kallisto = "/home/laura/projects/kallisto/build/src/kallisto"
bustools = "/home/laura/projects/bustools/build/src/bustools"

In [36]:
mut_file = "/home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698539_2_extracted_u10.fa"
mut_name = "0_ebov_snp_0_ms"
    
!mkdir -p $kallisto_out/$mut_name

!$kallisto bus \
    -i $ebola_index \
    -o $kallisto_out/$mut_name \
    -x bulk \
    -t 32 \
    $mut_file

!$bustools sort -o $kallisto_out/$mut_name/output_sorted.bus $kallisto_out/$mut_name/output.bus

!$bustools count \
    --genecounts \
    --cm \
    -o $kallisto_out/$mut_name/bustools_count/ \
    -g $ebola_t2g \
    -e $kallisto_out/$mut_name/matrix.ec \
    -t $kallisto_out/$mut_name/transcripts.txt \
    $kallisto_out/$mut_name/output_sorted.bus


[index] k-mer length: 31
[index] number of targets: 38
[index] number of k-mers: 17,819
[quant] running in single-end mode
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698539_2_extracted_u10.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 662 reads pseudoaligned

 all fits in buffer
Read in 662 BUS records
reading time 0s
sorting time 0s
writing time 0s


In [38]:
for mut_file in glob.glob(f"{out}/*_ms.fa"):
    mut_name = mut_file.split("/")[-1].split(".fa")[0]
    
    !mkdir -p $kallisto_out/$mut_name
    
    !$kallisto bus \
        -i $ebola_index \
        -o $kallisto_out/$mut_name \
        -x bulk \
        -t 32 \
        $mut_file

    !$bustools sort -o $kallisto_out/$mut_name/output_sorted.bus $kallisto_out/$mut_name/output.bus

    !$bustools count \
        --genecounts \
        --cm \
        -o $kallisto_out/$mut_name/bustools_count/ \
        -g $ebola_t2g \
        -e $kallisto_out/$mut_name/matrix.ec \
        -t $kallisto_out/$mut_name/transcripts.txt \
        $kallisto_out/$mut_name/output_sorted.bus


[index] k-mer length: 31
[index] number of targets: 38
[index] number of k-mers: 17,819
[quant] running in single-end mode
[quant] will process file 1: /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/mutated_files/3_ebov_snp_18_ms.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 7 reads pseudoaligned

 all fits in buffer
Read in 7 BUS records
reading time 0s
sorting time 0s
writing time 0s

[index] k-mer length: 31
[index] number of targets: 38
[index] number of k-mers: 17,819
[quant] running in single-end mode
[quant] will process file 1: /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/mutated_files/2_ebov_snp_08_ms.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 174 reads pseudoaligned

 all fits in buffer
Read in 174 BUS records
reading time 0s
sorting time 0s
writing time 0s

[index] k-mer length: 31
[index] number of targets: 38
[index] number of k-mers: 17,819
[qu